<a href="https://colab.research.google.com/github/hongkvu/Senior-Project-CMPE195A-B/blob/main/Diem'sNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Nov 19 04:01:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install tensorflow-gpu
!pip install einops
!pip install torch==1.12.1+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 588.3 MB 6.6 kB/s 
     |████████████████████████████████| 1.7 MB 69.4 MB/s 
     |████████████████████████████████| 439 kB 85.2 MB/s 
     |████████████████████████████████| 6.0 MB 58.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully 

## What is Einops?
Einops, an abbreviation of Einstein-Inspired Notation for operations is an open-source python framework for writing deep learning code in a new and better way. Einops provides us with new notation & new operations. It is  a flexible and powerful tool to ensure code readability and reliability with minimalist yet powerful API.

Supports numpy, pytorch, tensorflow, jax, and others.

Source: https://analyticsindiamag.com/reinventing-deep-learning-operation-via-einops/
##Example:
from einops import rearrange, reduce, repeat
### Rearrange elements according to the pattern
- output_tensor = rearrange(input_tensor, 't b c -> b c t')

### Combine rearrangement and reduction
- output_tensor = reduce(input_tensor, 'b c (h h2) (w w2) -> b h w c', 'mean', h2=2, w2=2)

### Copy along a new axis 
- output_tensor = repeat(input_tensor, 'h w -> h w c', c=3)

### Example given for einops, but code in other frameworks is almost identical  
````
from torch.nn import Sequential, Conv2d, MaxPool2d, Linear, ReLU
from einops.layers.torch import Rearrange

model = Sequential(
    Conv2d(3, 6, kernel_size=5),
    MaxPool2d(kernel_size=2),
    Conv2d(6, 16, kernel_size=5),
    MaxPool2d(kernel_size=2),
    # flattening
    Rearrange('b c h w -> b (c h w)'),  
    Linear(16*5*5, 120), 
    ReLU(),
    Linear(120, 10), 
)
````



CoAtNet src code


In [4]:
import torch
import torch.nn as nn

from einops import rearrange
from einops.layers.torch import Rearrange

# 	nn.Conv2d : Applies a 2D convolution over an input signal composed of several input planes.

def conv_3x3_bn(inp, oup, image_size, downsample=False):
    stride = 1 if downsample == False else 2
    return nn.Sequential(                               # build neural networks 
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),  # slide a matrix or filter over 2D data
        #Channel - The colors(?) that transmit the actual information to the receiver. Usually RGB. The colors in an image are created by a mix of Red, Green, and Blue.
        #inp = number of channels in the input image.
        #oup = number of channels produced by the convolution
        #3 = kernel size (3x3 in this case). Kernel is the filter used to extract the features from the images. The matrix that slides over the 2D data. (In 3D, a filter is a collection of kernels)
        #stride = how many pixels the kernel will shift over the image.
        #bias - is like the b parameter in y = mx + b -- Helps model fit the training set properly
        nn.BatchNorm2d(oup),  # Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs with additional channel dimension) 
        # Batch normalization normalizes the activations of the network between layers in batches so that the batches have a mean of 0 and a variance of 1
        nn.GELU() # The Gaussian Error Linear Unit is an activation function.
        # The GELU activation function is xΦ(x) 
        # GELU consistently achieves the lowest test error rate,
        # posing as a promising alternative to ReLU and ELU for a neural network’s activation function.
    )


class PreNorm(nn.Module):
    def __init__(self, dim, fn, norm):
        super().__init__()    # initializes the parent class object into the child class
        # __init__(): allows the class to initialize the attributes of the class
        # super(): function allows us to avoid using the base class name explicitly.
        self.norm = norm(dim)
        # self represents the instance of the class. By using the “self”  we can access the attributes and methods of the class in python. It binds the attributes with the given arguments.
        # norm is what is generally used to evaluate the error of a model
        # norm returns the matrix norm or vector norm of a given tensor
        # dim – Specifies which dimension or dimensions of input to calculate the norm across. 
        # If dim is None, the norm will be calculated across all dimensions of input. 
        # If the norm type indicated by p does not support the specified number of dimensions, an error will occur.
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)
        #**kwargs allow to pass multiple arguments or keyword arguments to a function. 


# SE Blocks - the Squeeze-Excite process:a modular implementation that can be easily implemented to most Deep CNNs.
# The main idea of an SE Block is: Assign each channel of a feature map a different weightage (excitation) based on how important each channel is (squeeze).
# crucial to the gains in performance
class SE(nn.Module):
    def __init__(self, inp, oup, expansion=0.25):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)                 #Sets the output size of the pooling layer to 1x1
        self.fc = nn.Sequential(
            nn.Linear(oup, int(inp * expansion), bias=False),   #applies a linear transformation
            nn.GELU(),        #applies the Gaussian Error Linear Units function
            nn.Linear(int(inp * expansion), oup, bias=False),
            nn.Sigmoid()      # An activation function that takes a value and turns it into a value between 0 and 1 to predict probabilities.
        )

    def forward(self, x):               # define the flow of forward process 
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c) # Squeeze - perform Global Average Pooling
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim), #applies a linear transformation
            nn.GELU(),       #applies the Gaussian Error Linear Units function
            nn.Dropout(dropout),  # takes in the dropout rate
            # to Prevent Neural Networks from Overfitting because it increase accuracy
            nn.Linear(hidden_dim, dim),   #applies a linear transformation
            nn.Dropout(dropout) # takes in the dropout rate
        )
    # define the flow of forward process 
    def forward(self, x):             # x in the forward() method is the input vector.
        return self.net(x)

# A MBConv is a Inverted Linear BottleNeck layer with Depth-Wise Separable Convolution.
class MBConv(nn.Module):
    def __init__(self, inp, oup, image_size, downsample=False, expansion=4):
        super().__init__()
        self.downsample = downsample    # reducing the sampling rate of a signal.
        # A reduction of the feature maps sizes(downsampling) 
        # as we move through the network enables the possibility of reducing the spatial resolution of the feature map.
        stride = 1 if self.downsample == False else 2
        # Stride is a parameter of the neural network's filter that modifies the amount of movement over the image or video. For example, if a neural network's stride is set to 1, the filter will move one pixel, or unit, at a time.
        #If stride = 2, the filter will move two pixels.
        hidden_dim = int(inp * expansion)   # Number of hidden layers self.
        # The hidden dimension is basically the number of nodes in each layer (like in the Multilayer Perceptron for example) 
        # The embedding size tells the size of the feature vector (the model uses embedded words as input)

        if self.downsample:
            self.pool = nn.MaxPool2d(3, 2, 1)   # Applies a 2D max pooling over an input signal composed of several input planes.
            # kernel_size=3, padding=2, dilation=1
            # kernel_size – the size of the window to take a max over
            # stride – the stride of the window. Default value is kernel_size
            # padding – implicit zero padding to be added on both sides
            # dilation – a parameter that controls the stride of elements in the window
            self.proj = nn.Conv2d(inp, oup, 1, 1, 0, bias=False)
            # slide a matrix or filter over 2D data

        if expansion == 1:
            self.conv = nn.Sequential(
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride,    # build neural networks that slide a matrix or filter over 2D data
                          1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),                     # Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs with additional channel dimension) 
                nn.GELU(),                                      #applies the Gaussian Error Linear Units function
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False), # build neural networks that slide a matrix or filter over 2D data
                nn.BatchNorm2d(oup),                             # Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs with additional channel dimension) 
            )
        else:
            self.conv = nn.Sequential(
                # pw
                # down-sample in the first conv
                nn.Conv2d(inp, hidden_dim, 1, stride, 0, bias=False),    # build neural networks that slide a matrix or filter over 2D data
                nn.BatchNorm2d(hidden_dim),                              # Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs with additional channel dimension)
                nn.GELU(),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, 1, 1,
                          groups=hidden_dim, bias=False),                # build neural networks that slide a matrix or filter over 2D data
                nn.BatchNorm2d(hidden_dim),                              # Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs with additional channel dimension)
                nn.GELU(),                                               #applies the Gaussian Error Linear Units function
                SE(inp, hidden_dim),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),         # build neural networks that slide a matrix or filter over 2D data
                nn.BatchNorm2d(oup),                                     # Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs with additional channel dimension)
            ) 
        
        self.conv = PreNorm(inp, self.conv, nn.BatchNorm2d)

    def forward(self, x):           # define the flow of forward process
        if self.downsample:
            return self.proj(self.pool(x)) + self.conv(x)
        else:
            return x + self.conv(x)

# multihead attention: The goal is to take an average over the features of multiple elements.
# Allows the model to jointly attend to information from different representation subspaces
# Attention models (or mechanisms) are neural network input processing strategies 
# that allow the network to focus on specific parts of a complicated input one by one until the entire dataset is categorized
class Attention(nn.Module):
    def __init__(self, inp, oup, image_size, heads=8, dim_head=32, dropout=0.):
      # dim_head – Total dimension of the model.
      # heads – Number of parallel attention heads. Note that dim_head will be split across num_heads (i.e. each head will have dimension dim_head // heads).
      # dropout – Dropout probability on attn_output_weights. Default: 0.0 (no dropout).
        super().__init__()
        inner_dim = dim_head * heads
        project_out = not (heads == 1 and dim_head == inp)

        self.ih, self.iw = image_size

        self.heads = heads
        self.scale = dim_head ** -0.5

        # parameter table of relative position bias
        self.relative_bias_table = nn.Parameter(
            torch.zeros((2 * self.ih - 1) * (2 * self.iw - 1), heads))

        coords = torch.meshgrid((torch.arange(self.ih), torch.arange(self.iw)))
        coords = torch.flatten(torch.stack(coords), 1)
        relative_coords = coords[:, :, None] - coords[:, None, :]

        relative_coords[0] += self.ih - 1
        relative_coords[1] += self.iw - 1
        relative_coords[0] *= 2 * self.iw - 1
        relative_coords = rearrange(relative_coords, 'c h w -> h w c')
        relative_index = relative_coords.sum(-1).flatten().unsqueeze(1)
        self.register_buffer("relative_index", relative_index)

        self.attend = nn.Softmax(dim=-1)
        self.to_qkv = nn.Linear(inp, inner_dim * 3, bias=False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, oup),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(
            t, 'b n (h d) -> b h n d', h=self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        # Use "gather" for more efficiency on GPUs
        relative_bias = self.relative_bias_table.gather(
            0, self.relative_index.repeat(1, self.heads))
        relative_bias = rearrange(
            relative_bias, '(h w) c -> 1 c h w', h=self.ih*self.iw, w=self.ih*self.iw)
        dots = dots + relative_bias

        attn = self.attend(dots)
        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        out = self.to_out(out)
        return out

# Transformer architecture implements an encoder-decoder structure without recurrence and convolutions in order to generate an output.
# A transformer is a deep learning model that adopts the mechanism of self-attention, differentially weighting the significance of each part of the input data.
class Transformer(nn.Module):
    def __init__(self, inp, oup, image_size, heads=8, dim_head=32, downsample=False, dropout=0.):
        super().__init__()
        hidden_dim = int(inp * 4)

        self.ih, self.iw = image_size
        self.downsample = downsample

        if self.downsample:
            self.pool1 = nn.MaxPool2d(3, 2, 1)
            self.pool2 = nn.MaxPool2d(3, 2, 1)
            self.proj = nn.Conv2d(inp, oup, 1, 1, 0, bias=False)

        self.attn = Attention(inp, oup, image_size, heads, dim_head, dropout)     # attention technique
        #inp = number of channels in the input image.
        #oup = number of channels produced by the convolution
        #dim = dimension per head
        self.ff = FeedForward(oup, hidden_dim, dropout)
        # feed-forward neural networks: used in practice to transform the attention vectors into a digestible form by the next encoder/decoder block

        self.attn = nn.Sequential(
            Rearrange('b c ih iw -> b (ih iw) c'),
            PreNorm(inp, self.attn, nn.LayerNorm),
            Rearrange('b (ih iw) c -> b c ih iw', ih=self.ih, iw=self.iw)
        )

        self.ff = nn.Sequential(
            Rearrange('b c ih iw -> b (ih iw) c'),  # # concatenated images along horizontal axis
            # einops.rearrange is a reader-friendly smart element reordering for multidimensional tensors. 
            # This operation includes functionality of transpose (axes permutation), reshape (view), squeeze, unsqueeze, stack, concatenate and other operations.
            PreNorm(oup, self.ff, nn.LayerNorm),
            # nn.LayerNorm - Applies Layer Normalization over a mini-batch of inputs as described in the paper Layer Normalization
            Rearrange('b (ih iw) c -> b c ih iw', ih=self.ih, iw=self.iw)   # space-to-depth operation
            # ih, iw = image_size
        )

    def forward(self, x):
        if self.downsample:
            x = self.proj(self.pool1(x)) + self.attn(self.pool2(x))
        else:
            x = x + self.attn(x)
        x = x + self.ff(x)
        return x


class CoAtNet(nn.Module):
    def __init__(self, image_size, in_channels, num_blocks, channels, num_classes=1000, block_types=['C', 'C', 'T', 'T']):
        super().__init__()
        ih, iw = image_size
        block = {'C': MBConv, 'T': Transformer}

        self.s0 = self._make_layer(
            conv_3x3_bn, in_channels, channels[0], num_blocks[0], (ih // 2, iw // 2))
        self.s1 = self._make_layer(
            block[block_types[0]], channels[0], channels[1], num_blocks[1], (ih // 4, iw // 4))
        self.s2 = self._make_layer(
            block[block_types[1]], channels[1], channels[2], num_blocks[2], (ih // 8, iw // 8))
        self.s3 = self._make_layer(
            block[block_types[2]], channels[2], channels[3], num_blocks[3], (ih // 16, iw // 16))
        self.s4 = self._make_layer(
            block[block_types[3]], channels[3], channels[4], num_blocks[4], (ih // 32, iw // 32))

        self.pool = nn.AvgPool2d(ih // 32, 1)
        self.fc = nn.Linear(channels[-1], num_classes, bias=False)

    def forward(self, x):
        x = self.s0(x)
        x = self.s1(x)
        x = self.s2(x)
        x = self.s3(x)
        x = self.s4(x)

        x = self.pool(x).view(-1, x.shape[1])
        x = self.fc(x)
        return x

    def _make_layer(self, block, inp, oup, depth, image_size):
        layers = nn.ModuleList([])
        for i in range(depth):
            if i == 0:
                layers.append(block(inp, oup, image_size, downsample=True))
            else:
                layers.append(block(oup, oup, image_size))
        return nn.Sequential(*layers)


def coatnet_0():
    num_blocks = [2, 2, 3, 5, 2]            # L
    channels = [64, 96, 192, 384, 768]      # D
    return CoAtNet((224, 224), 3, num_blocks, channels, num_classes=1000)


def coatnet_1():
    num_blocks = [2, 2, 6, 14, 2]           # L
    channels = [64, 96, 192, 384, 768]      # D
    return CoAtNet((224, 224), 3, num_blocks, channels, num_classes=1000)


def coatnet_2():
    num_blocks = [2, 2, 6, 14, 2]           # L
    channels = [128, 128, 256, 512, 1026]   # D
    return CoAtNet((224, 224), 3, num_blocks, channels, num_classes=1000)


def coatnet_3():
    num_blocks = [2, 2, 6, 14, 2]           # L
    channels = [192, 192, 384, 768, 1536]   # D
    return CoAtNet((224, 224), 3, num_blocks, channels, num_classes=1000)


def coatnet_4():
    num_blocks = [2, 2, 12, 28, 2]          # L
    channels = [192, 192, 384, 768, 1536]   # D
    return CoAtNet((224, 224), 3, num_blocks, channels, num_classes=1000)



#PyTorch doesn't have a function to calculate the total number of parameters as Keras does, 
#but it's possible to sum the number of elements for every parameter group
#trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


if __name__ == '__main__':
    img = torch.randn(1, 3, 224, 224)

    net = coatnet_0()
    out = net(img)
    print(out.shape, count_parameters(net))

    net = coatnet_1()
    out = net(img)
    print(out.shape, count_parameters(net))

    net = coatnet_2()
    out = net(img)
    print(out.shape, count_parameters(net))

    net = coatnet_3()
    out = net(img)
    print(out.shape, count_parameters(net))

    net = coatnet_4()
    out = net(img)
    print(out.shape, count_parameters(net))

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


torch.Size([1, 1000]) 17789624
torch.Size([1, 1000]) 33170624
torch.Size([1, 1000]) 55767564
torch.Size([1, 1000]) 117724480
torch.Size([1, 1000]) 203960368


Pytorch

Pytorch is an open source machine learning framework that accelerates the path from research prototyping to production deployment.

- Pytorch.nn: The nn package defines a set of Modules, which you can think of as a neural network layer that has produces output from input and may have some trainable weights.
  - nn documents: https://pytorch.org/docs/stable/nn.html
  - Container 
    - nn.Module: Base class for all neural network modules.
      - Example: 
            class Model(nn.Module):
              def __init__(self):
              super().__init__()
              self.conv1 = nn.Conv2d(1, 20, 5)
              self.conv2 = nn.Conv2d(20, 20, 5)
      - Modules can also contain other Modules, allowing to nest them in a tree structure: 
            module (Module) – child module to be added to the module
    - nn.Sequential: A sequential container.
      - Class torch.nn.Sequential(*args) 
      - Modules will be added to it in the order they are passed in the constructor. It accepts any input and forwards it to the first module it contains. It then “chains” outputs to inputs sequentially for each subsequent module, finally returning the output of the last module.
      - Example: 
            model = nn.Sequential(
                      nn.Conv2d(1,20,5),
                      nn.ReLU(),
                      nn.Conv2d(20,64,5),
                      nn.ReLU()
                    )
      -> Run `model`: input will first be passed to `Conv2d(1,20,5)` -> output of `Conv2d(1,20,5)` is used as input to the first `ReLU` -> output of the first `ReLU` becomes input for `Conv2d(20,64,5)` -> output of `Conv2d(20,64,5)` is used as input to the second `ReLU`.
  - Convolution Layers (https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md)
    - nn.Conv2d: Applies a 2D convolution over an input signal composed of several input planes.
    - Constructor: torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
      - stride: stride for the cross-correlation, a single number or a tuple.
      - padding: amount of padding applied to the input. 
      - dilation: spacing between the kernel points.
      - groups: connections between inputs and outputs.
        - 1: all inputs are convolved to all outputs.
        - 2: the operation becomes equivalent to having two conv layers side by side, each seeing half the input channels and producing half the output channels, and both subsequently concatenated.
        - in_channels: each input channel is convolved with its own set of filters  
  - Non-linear Activations
    - nn.ReLU: Applies the rectified linear unit function element-wise: 
            ReLU(x) = (x)+ = max(0,x)
      - Constructor: CLASStorch.nn.ReLU(inplace=False)
        - inplace – can optionally do the operation in-place. Default: False
  - Normalization Layers
    - nn.BatchNorm2d: Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs with additional channel dimension).
    - Constructor: CLASS torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)
      - num_features – CC from an expected input of size (N, C, H, W)(N,C,H,W)
      - eps – a value added to the denominator for numerical stability.
        - Default: 1e-5
      - momentum – the value used for the running_mean and running_var computation. 
        - None for cumulative moving average 
        - Default: 0.1
      - affine – a boolean value. 
        - True: this module has learnable affine parameters. Default: True
      - track_running_stats – a boolean value. Default: True
        - True: this module tracks the running mean and variance,
        - False: this module does not track such statistics 
    - read: https://arxiv.org/abs/1502.03167


- torch.randn (*size, *, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False)
  - Returns a tensor filled with random numbers from a normal distribution with mean 0 and variance 1 (standard normal distribution).
  - Parameters
    - size - sequence of integers defining the size of the output tensor. Can be a variable number of arguments or a collection like a list or tuple.
    - out (Tensor, optional) – the output tensor.
    - dtype (torch.dtype, optional) – the desired data type of returned tensor. 
      - Default: None - uses a global default.
    - layout (torch.layout, optional) – the desired layout of returned Tensor. 
      - Default: torch.strided.
    - device (torch.device, optional) 
      - the desired device of returned tensor. 
      - Default: None - uses the current device for the default tensor type. device will be the CPU for CPU tensor types and the current CUDA device for CUDA tensor types.
    - requires_grad (bool, optional) – If autograd should record operations on the returned tensor. Default: False.

In [5]:
import torch
x=torch.randn(1,3,224,224)
print(x)
x.shape

tensor([[[[ 1.7537, -0.3105, -1.3423,  ...,  0.5359,  0.9673, -0.7530],
          [ 0.6211, -0.5043,  0.9315,  ...,  1.0312, -0.8310,  0.3585],
          [ 0.7764,  0.7485,  0.3152,  ..., -0.6898, -0.4531, -0.8433],
          ...,
          [ 0.7117, -1.5021, -0.6562,  ...,  1.2093,  0.1014,  0.6339],
          [ 0.5769,  0.7071,  1.7586,  ...,  1.0662, -1.4365, -1.8449],
          [-1.0990, -0.3439, -1.2022,  ..., -1.0464, -0.8165,  1.3293]],

         [[-1.2342, -0.5265,  1.6963,  ...,  2.2013, -0.2321, -0.5455],
          [-0.1278,  0.4351,  0.0101,  ..., -0.7596, -0.2701,  0.0782],
          [-0.9278,  0.4251,  1.4075,  ..., -2.3347,  0.0717,  0.0510],
          ...,
          [-0.3132,  0.3478, -1.7379,  ...,  0.5018,  0.9478, -0.1219],
          [ 0.7730, -0.4916,  0.4235,  ...,  0.2819,  0.7906,  0.3742],
          [ 0.1550,  1.7120,  0.6051,  ..., -0.6804, -1.8915,  1.9394]],

         [[ 1.0537,  0.1272,  0.9023,  ..., -1.3795, -0.5792, -0.8099],
          [ 1.6814,  0.6962,  

torch.Size([1, 3, 224, 224])

In [6]:
import torch
x=torch.randn(2,4)
print(x)
x.shape

tensor([[ 0.3749, -1.5610,  1.4717,  1.2633],
        [-1.5209, -1.1058,  0.1023, -0.5608]])


torch.Size([2, 4])

In [7]:
import torch
x=torch.randn(2,4,5)
print(x)
x.shape

tensor([[[ 0.3388,  0.8788, -1.2196, -1.2401, -0.0170],
         [ 0.4854,  0.1056,  0.4515, -0.5352, -0.7042],
         [ 0.4569,  0.2283, -0.1194, -0.7109, -1.3875],
         [-0.3036,  0.3397,  0.0556, -0.8855,  1.1138]],

        [[-1.0179, -0.8005,  0.7045,  0.6460, -0.3495],
         [ 0.6218,  0.7136,  0.0900, -0.3309,  1.0665],
         [-0.0162,  0.4148, -0.7192, -0.5895, -0.9754],
         [-1.3970,  0.3525, -2.9724,  1.4578,  0.0604]]])


torch.Size([2, 4, 5])



```
def conv_3x3_bn(inp, oup, image_size, downsample=False): stride = 1 if downsample == False else 2
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.GELU()
    )
```



**nn.Sequential()**

Class that allows us to build neural networks on the fly without having to define an explicit class.

  **nn.Conv2d(inp, oup, 3, stride, 1, bias=False)**

  2D Convolution - We slide a matrix or filter over 2D data (an image turned into a grid of numbers) and perform element-wise multiplication with the data, then sum up the multiplication result to produce an output. We move the filter in strides until we get the final output matrix of the 2D convolution operation.

  Channel - The colors(?) that transmit the actual information to the receiver. Usually RGB. The colors in an image are created by a mix of Red, Green, and Blue.

  inp = number of channels in the input image. 

  oup = number of channels produced by the convolution

  3 = kernel size (3x3 in this case). Kernel is the filter used to extract the features from the images. The matrix that slides over the 2D data. (In 3D, a filter is a collection of kernels)

  stride = how many pixels the kernel will shift over the image.
  
  bias - is like the *b* parameter in *y = mx + b* -- Helps model fit the training set properly

  **nn.BatchNorm2d(oup)** 
  is needed to normalize the output so the data will be on the same scale. If one feature is too large, this feature will drown out the smaller feature. During gradient descent, the neural network will have to make a large update to one weight compared to the other weight. It can cause the gradient descent trajectory to oscillate back and forth, thus taking more steps to reach the minimum.

  **nn.GELU**

  Gaussian Error Linear Units function. An activation function like sigmoid and RELU



---

```
class SE(nn.Module):
    def __init__(self, inp, oup, expansion=0.25):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(oup, int(inp * expansion), bias=False),
            nn.GELU(),
            nn.Linear(int(inp * expansion), oup, bias=False),
            nn.Sigmoid()
        )
```

**nn.AdaptiveAvgPool2d(1)**

Sets the output size of the pooling layer to the specified size. (1x1 in this case) The stride and kernel-size are automatically selected to get this output size.

Pooling compresses and generalizes the features in the feature map.

**nn.Linear()**

**nn.Sigmoid()**

An activation function that takes a value and turns it into a value between 0 and 1 to predict probabilities.
  




Import libraries

In [8]:
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [9]:
import torch.optim as optim #optimizer, will hold the current state and will update the parameters based on the computed gradients
import torch
import torch.nn as nn #neural network framework
import torch.nn.parallel 
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms #common image transformation
from torch.autograd import Variable #automatic differentiation

Set global parameters

In [10]:
modellr = 1e-4
BATCH_SIZE = 16 #the dataset is divided into batches, each of 16 
EPOCHS = 10 #number of times the entire dataset is passed through the neural network
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')#the GPU device that operations will be on

Data Preprocessing

In [11]:
#modifying images (resize, convert, normalize)  
transform = transforms.Compose([ #chain the transformation together
    transforms.Resize((224, 224)), #resize image
    transforms.ToTensor(), #convert image to a tensor (matrix)
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) #normalize with (mean, standard deviation)

])
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

Fetch the data

In [12]:
# coding:utf8
import os
from PIL import Image
from torch.utils import data
from torchvision import transforms as T
from sklearn.model_selection import train_test_split
 
# Labels = {'alpine_sea_holly': 0, 'alstroemeria': 1, 'amaranth': 2, 'amaryllis': 3, 'ammi': 4, 'anemone': 5, 
#           'anthurium': 6, 'antirrhinum': 7, 'artichoke': 8, 'asian virginsbower': 9, 'aster': 10,
#           'astilbe': 11, 'astrantia': 12, 'ball moss': 13, 'ballon flower': 14, 'barbeton daisy': 15,
#           'bird of paradise': 16, 'black-eye': 17, 'bluebell': 18, 'bougainvillea': 19, 'bouvardia': 20,
#           'brassica': 21, 'buttercup': 22, 'california poppy': 23, 'calla': 24, 'canna lily': 25,
#           'carnation': 26, 'celosia': 27, 'chamelaucium': 28, 'chamomile': 29, 'chrysanthemum': 30,
#           'colts_foot': 31, 'columbine': 32, 'cotton': 33, 'cowslip': 34, 'craspedia': 35,
#           'crocus': 36, 'cyclamen': 37, 'daffodil': 38, 'dahlia': 39, 'daisy':40,
#           'dandelion':41, 'delphinium': 42, 'eryngium': 43, 'eustoma': 44, 'forget-me-not': 45,
#           'foxglove': 46, 'frangipani': 47, 'freesia': 48, 'fritillary': 49, 'garden phlox': 50,
#           'gerbera': 51, 'gladiolus': 52, 'globe thistle': 53, 'gypsophila': 54, 'hyacinth': 55,
#           'hydrangea': 56, 'hypericum': 57, 'iris': 58, 'lavender': 59, 'leucadendron': 60,
#           'leucospermum': 61, 'lilac': 62, 'lily': 63, 'lily_of_the_valley': 64, 'lily_valley': 65,
#           'limonium': 66, 'lotus': 67, 'love in the mist': 68, 'mattiola': 69, 'mimosa': 70,
#           'morning glory': 71, 'muscari': 72, 'narcissus': 73, 'nelumbo nucifera': 74, 'orchid': 75,
#           'ornithogalum': 76, 'oxeye daisy': 77, 'oxypetalum': 78, 'ozotamnus': 79, 'pansy': 80,
#           'passion flower': 81, 'peony': 82, 'petunia': 83, 'poinsettia': 84, 'protea': 85,
#           'ranunculus': 86, 'rose': 87, 'sedum': 88, 'silverbush': 89, 'skimmia': 90,
#           'snowdrop': 91, 'solidago': 92, 'strelitzia': 93, 'sunflower': 94, 'tigerlily': 95,
#           'trachelium': 96, 'tropical white morning-glory': 97, 'tulip': 98, 'veronica': 99, 'water lily': 100,
#           'willow': 101, 'windflower': 102}

# Labels = {
#      'tulip': 0, 'sunflower': 1, 'rose': 2, 'dandelion': 3, 'daisy':4
# }
Labels = {
     'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip':4
}
 
class FlowerData (data.Dataset):
 
    def __init__(self, root, transforms=None, train=True, test=False):
        """
        Main objective: to obtain the addresses of all pictures and divide the data according to training, verification and test
        """
        self.test = test
        self.transforms = transforms
 
        if self.test:
            imgs = [os.path.join(root, img) for img in os.listdir(root)]
            self.imgs = imgs
        else:
            imgs_labels = [os.path.join(root, img) for img in os.listdir(root)]
            imgs = []
            for imglable in imgs_labels:
                for imgname in os.listdir(imglable):
                    imgpath = os.path.join(imglable, imgname)
                    imgs.append(imgpath)
            trainval_files, val_files = train_test_split(imgs, test_size=0.3, random_state=42)
            if train:
                self.imgs = trainval_files
            else:
                self.imgs = val_files
 
    def __getitem__(self, index):
        """
        Returns the data of one picture at a time
        """
        img_path = self.imgs[index]
        img_path=img_path.replace("\\",'/')
        if self.test:
            label = -1
        else:
            labelname = img_path.split('/')[-2]
            label = Labels[labelname]
        data = Image.open(img_path).convert('RGB')
        data = self.transforms(data)
        return data, label
 
    def __len__(self):
        return len(self.imgs)

In [13]:
# split-data
!pip install split-folders
import splitfolders
import pathlib
data = "drive/My Drive/195A+BSeniorProjectGroupWorks/Datasets/Flower_Dataset/flowers"
data_splitted = "drive/My Drive/195A+BSeniorProjectGroupWorks/Datasets/Flower_Dataset/flowers_train_test_set"
# splitfolders.ratio(data, data_splitted, seed=1337, ratio=(.8, .1, .1)) 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
# Read data
# Suggestion: automates image look-up/download from the web
dataset_train = FlowerData(pathlib.Path(data_splitted)/"train", transforms=transform, train=True)
dataset_test = FlowerData(pathlib.Path(data_splitted)/"test", transforms=transform_test, train=False)
# from sklearn.model_selection import train_test_split
# x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
# dataset_train, dataset_test = train_test_split(df, test_size=0.2, random_state=25)

In [15]:
# Import data
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)

Set model

In [16]:
# Instantiate the model and move to the GPU
criterion = nn.CrossEntropyLoss()

model_ft = coatnet_1()
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 200) #change the 2nd arg according to the number of Labels
#TODO: look into why changing this arg fixes the error
model_ft.to(DEVICE)#move the model to GPU
# Choose the simple and violent Adam optimizer to reduce the learning rate
optimizer = optim.Adam(model_ft.parameters(), lr=modellr)
cosine_schedule = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer,T_max=20,eta_min=1e-9)

In [17]:
# Define training process

def train(model, device, train_loader, optimizer, epoch):
    model.train()#set the mode to train mode, not actually training using this function
    sum_loss = 0
    total_num = len(train_loader.dataset)
    print(total_num, len(train_loader))
    for batch_idx, (data, target) in enumerate(train_loader):#enumrate is a counter for the loop
        data, target = Variable(data).to(device), Variable(target).to(device)#send data
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print_loss = loss.data.item()
        sum_loss += print_loss
        if (batch_idx + 1) % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                       100. * (batch_idx + 1) / len(train_loader), loss.item()))
    ave_loss = sum_loss / len(train_loader)
    print('epoch:{},loss:{}'.format(epoch, ave_loss))


# Verification process
def val(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total_num = len(test_loader.dataset)
    print(total_num, len(test_loader))
    with torch.no_grad():
        for data, target in test_loader:
            data, target = Variable(data).to(device), Variable(target).to(device)
            output = model(data)
            loss = criterion(output, target)
            _, pred = torch.max(output.data, 1)
            correct += torch.sum(pred == target)
            print_loss = loss.data.item()
            test_loss += print_loss
        correct = correct.data.item()
        acc = correct / total_num
        avgloss = test_loss / len(test_loader)
        print('\nVal set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            avgloss, correct, len(test_loader.dataset), 100 * acc))


# train

for epoch in range(1, EPOCHS + 1):
    train(model_ft, DEVICE, train_loader, optimizer, epoch)
    cosine_schedule.step()
    val(model_ft, DEVICE, test_loader)


2444 153
Train Epoch: 1 [160/2444 (7%)]	Loss: 2.285874
Train Epoch: 1 [320/2444 (13%)]	Loss: 1.512222
Train Epoch: 1 [480/2444 (20%)]	Loss: 1.328996
Train Epoch: 1 [640/2444 (26%)]	Loss: 0.974019
Train Epoch: 1 [800/2444 (33%)]	Loss: 1.555632
Train Epoch: 1 [960/2444 (39%)]	Loss: 1.124369
Train Epoch: 1 [1120/2444 (46%)]	Loss: 1.177095
Train Epoch: 1 [1280/2444 (52%)]	Loss: 1.288607
Train Epoch: 1 [1440/2444 (59%)]	Loss: 1.853799
Train Epoch: 1 [1600/2444 (65%)]	Loss: 1.438674
Train Epoch: 1 [1760/2444 (72%)]	Loss: 1.435918
Train Epoch: 1 [1920/2444 (78%)]	Loss: 1.043680
Train Epoch: 1 [2080/2444 (85%)]	Loss: 1.135460
Train Epoch: 1 [2240/2444 (92%)]	Loss: 1.401072
Train Epoch: 1 [2400/2444 (98%)]	Loss: 1.095912
epoch:1,loss:1.4211071480333415
132 9

Val set: Average loss: 1.0822, Accuracy: 74/132 (56%)

2444 153
Train Epoch: 2 [160/2444 (7%)]	Loss: 1.933193
Train Epoch: 2 [320/2444 (13%)]	Loss: 0.782915
Train Epoch: 2 [480/2444 (20%)]	Loss: 1.203809
Train Epoch: 2 [640/2444 (26%)]	Los

In [18]:
torch.save(model_ft, 'model.pth')

Test

In [19]:
classes = (
     'daisy', 'dandelion', 'rose', 'sunflower', 'tulip'
)
     
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [20]:
#load the model and put model in DEVICE
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load("model.pth")
model.eval()
model.to(DEVICE)

CoAtNet(
  (s0): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU(approximate=none)
    )
    (1): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU(approximate=none)
    )
  )
  (s1): Sequential(
    (0): MBConv(
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (proj): Conv2d(64, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (conv): PreNorm(
        (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (fn): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affi

In [21]:
#Read picture and predict category of picture
path = 'drive/MyDrive/195A+BSeniorProjectGroupWorks/Datasets/test(christy)/'
testList = os.listdir(path)
for file in testList:
    img = Image.open(path + file)
    img = transform_test(img)
    img.unsqueeze_(0)
    img = Variable(img).to(DEVICE)
    out = model(img)
    # Predict
    _, pred = torch.max(out.data, 1)
    print('Image Name: {}, predict: {}'.format(file, classes[pred.data.item()]))

Image Name: daisy.jpg, predict: daisy
Image Name: rose.jpg, predict: rose
Image Name: sunflower.jpg, predict: sunflower
Image Name: tulip.jpg, predict: tulip
Image Name: daisy2.jpg, predict: daisy
Image Name: daisy3.jpg, predict: daisy
Image Name: rose2.jpg, predict: tulip
Image Name: rose3.jpg, predict: rose
Image Name: sunflower2.jpg, predict: sunflower
Image Name: sunflower3.jpg, predict: sunflower
Image Name: tulip2.jpg, predict: tulip
Image Name: tulip3.jpg, predict: tulip
Image Name: dandelion.jpg, predict: dandelion
Image Name: dandelion2.jpg, predict: dandelion
Image Name: dandelion3.jpg, predict: tulip
Image Name: sunflower_dl.jpeg, predict: sunflower


Note: larger dataset yields worse result
Why?
*Figure this out and make it a novelty point* 

Priority: expand dataset

In [ ]:
# Show input and output of a directly uploaded picture
from google.colab import files
uploaded = files.upload()
# Show the image
from matplotlib import pyplot as plt
input = plt.imread("")
img = plt.imshow(input)
# Resize the image
from skimage.transform import resize
resized_img = resize(input, (32,32,3))
img = plt.imshow(resized_img)
# Get model predictions
img = transform_test(img)
img.unsqueeze_(0)
out = model(img)
_, pred = torch.max(out.data, 1)
print('Image Name: {}, predict: {}'.format(file, classes[pred.data.item()]))
